In [ ]:
import numpy as np
import pandas as pd
import scipy.stats
import plotly.express as px
import plotly.graph_objects as pg
from scipy.optimize import curve_fit

import dt4dds.analysis.dataaggregation as analysis

import sys
sys.path.append('..')
import plotting

# Coverage bias

In [ ]:
data = analysis.DistributionAnalysis({
    'Lietard_Normal': "../data_experimental/Photolithographic_Lietard/normal/analysis/coverage.tsv",
    'Lietard_Capped': "../data_experimental/Photolithographic_Lietard/capped/analysis/coverage.tsv",
    'Lietard_Spaced': "../data_experimental/Photolithographic_Lietard/spaced/analysis/coverage.tsv",
    'Lietard_highdensity': "../data_experimental/Photolithographic_Lietard/high_density/analysis/coverage.tsv",
    'Antkowiak_File1': "../data_experimental/Photolithographic_Antkowiak/File1/analysis/coverage.tsv",
    'Antkowiak_File2': "../data_experimental/Photolithographic_Antkowiak/File2/analysis/coverage.tsv",
    'Antkowiak_File3': "../data_experimental/Photolithographic_Antkowiak/File3/analysis/coverage.tsv",
})
order = ['Lietard_Normal', 'Lietard_Capped', 'Lietard_Spaced', 'Lietard_highdensity', 'Antkowiak_File1', 'Antkowiak_File2', 'Antkowiak_File3']

In [ ]:
plot_data = data.data.copy()

fig = px.histogram(
    plot_data, 
    x="x", 
    facet_col="exp",
    facet_col_wrap=4, 
    facet_col_spacing=0.05,
    facet_row_spacing=0.1,
    barmode='overlay',
    color_discrete_sequence=['#6baed6'],
    range_x=[0, 2.5], 
    range_y=[0, 1.5], 
    opacity=0.65,
    nbins=1000,
    histnorm='probability density'
)
fig.update_traces(xbins={'start': 0.0, 'end': 3, 'size': 0.1}, selector=dict(type='histogram'))
fig.for_each_annotation(lambda a: a.update(text=''))


fig.for_each_yaxis(lambda yaxis: yaxis.update(dtick=1))
fig.update_traces(marker=dict(line_width=0), selector=dict(type='histogram')) 

fig.update_layout(
    template="simple_white", 
    height=300, 
    width=680, 
    showlegend=False, 
    margin=dict(l=0, r=10, t=10, b=0),
    font_family="Inter",
    legend_font_size=28/3,
)
fig.update_xaxes(
    title_text='',
    title_font_family="Inter", 
    title_font_size=28/3, 
    dtick=1,
    minor_ticks="outside", 
    minor_dtick=0.5,
    tickfont_size=28/3, 
)
fig.update_xaxes(
    title_text='Normalized coverage', row=1
)
fig.update_yaxes(
    title_text='Probability density',
    title_font_family="Inter", 
    title_font_size=28/3, 
    dtick=1,
    minor_ticks="outside", 
    minor_dtick=0.5,
    tickfont_size=28/3, 
    col=1
)

fig.show()
fig.write_image("SI_figures/coverage_histograms.svg")

# Plot only one histogram

In [ ]:
data = analysis.DistributionAnalysis({
    'Antkowiak_File1': "../data_experimental/Photolithographic_Antkowiak/File1/analysis/coverage.tsv",
})

In [ ]:
counts36 = np.load("./run36.npy")
density, bins = np.histogram(counts36, bins=np.arange(0, 200, 1), density=True)
bins = 0.5*(bins[:-1] + bins[1:])
bins /= counts36.mean() # convert to relative coverage
ref_stepsize = 1/counts36.mean()

In [ ]:
plot_data = data.data.copy()

fig = px.histogram(
    plot_data, 
    x="x", 
    barmode='overlay',
    color_discrete_sequence=['#999999'],
    range_x=[0, 2.5], 
    range_y=[0, 0.13], 
    opacity=1,
    nbins=1000,
    facet_row_spacing=0.05,
    histnorm='probability'
)
fig.update_traces(xbins={'start': 0.0, 'end': 3, 'size': 0.1}, selector=dict(type='histogram'))
fig.for_each_annotation(lambda a: a.update(text=''))

fig.add_trace(pg.Scatter(
    x=bins, 
    y=density/ref_stepsize*0.1, # we correct the density to match the histogram
    mode="lines", 
    line=dict(color="black", width=1, dash="dot")),
)

fig.for_each_yaxis(lambda yaxis: yaxis.update(dtick=1))
fig.update_traces(marker=dict(line_width=0), selector=dict(type='histogram')) 

fig.update_layout(
    template="simple_white", 
    height=175, 
    width=200, 
    showlegend=False, 
    margin=dict(l=0, r=10, t=10, b=0),
    font_family="Inter",
    legend_font_size=28/3,
)
fig.update_xaxes(
    title_text='',
    title_font_family="Inter", 
    title_font_size=28/3, 
    dtick=1,
    minor_ticks="outside", 
    minor_dtick=0.5,
    tickfont_size=28/3, 
)
fig.update_xaxes(
    title_text='Normalized coverage', row=1, col=1
)
fig.update_yaxes(
    title_text='Fraction of design sequences',
    title_font_family="Inter", 
    title_font_size=28/3, 
    dtick=0.05,
    tickformat=',.0%',
    minor_ticks="outside", 
    minor_dtick=0.01,
    tickfont_size=28/3, 
)

fig.show()
fig.write_image("./figures/coverage_histogram.svg")

### Lognormal std for coverage bias

In [ ]:
np.std(np.log(plot_data.x))

# Plot only simulation

In [ ]:
data = analysis.DistributionAnalysis({
    'simulated_simulated': "../data_simulated/test_photolithography/analysis/coverage.tsv",
})

In [ ]:
counts36 = np.load("./run36.npy")
density, bins = np.histogram(counts36, bins=np.arange(0, 200, 1), density=True)
bins = 0.5*(bins[:-1] + bins[1:])
bins /= counts36.mean() # convert to relative coverage
ref_stepsize = 1/counts36.mean()

In [ ]:
plot_data = data.data.copy()

fig = px.histogram(
    plot_data, 
    x="x", 
    barmode='overlay',
    color_discrete_sequence=['#999999'],
    range_x=[0, 2.5], 
    range_y=[0, 0.13], 
    opacity=1,
    nbins=1000,
    facet_row_spacing=0.05,
    histnorm='probability'
)
fig.update_traces(xbins={'start': 0.0, 'end': 3, 'size': 0.1}, selector=dict(type='histogram'))
fig.for_each_annotation(lambda a: a.update(text=''))

fig.add_trace(pg.Scatter(
    x=bins, 
    y=density/ref_stepsize*0.1, # we correct the density to match the histogram
    mode="lines", 
    line=dict(color="black", width=1, dash="dot")),
)

fig.for_each_yaxis(lambda yaxis: yaxis.update(dtick=1))
fig.update_traces(marker=dict(line_width=0), selector=dict(type='histogram')) 

fig.update_layout(
    template="simple_white", 
    height=175, 
    width=200, 
    showlegend=False, 
    margin=dict(l=0, r=10, t=10, b=0),
    font_family="Inter",
    legend_font_size=28/3,
)
fig.update_xaxes(
    title_text='',
    title_font_family="Inter", 
    title_font_size=28/3, 
    dtick=1,
    minor_ticks="outside", 
    minor_dtick=0.5,
    tickfont_size=28/3, 
)
fig.update_xaxes(
    title_text='Normalized coverage', row=1, col=1
)
fig.update_yaxes(
    title_text='Fraction of design sequences',
    title_font_family="Inter", 
    title_font_size=28/3, 
    dtick=0.05,
    tickformat=',.0%',
    minor_ticks="outside", 
    minor_dtick=0.01,
    tickfont_size=28/3, 
)

fig.show()
fig.write_image("./SI_figures/coverage_histogram_simulation.svg")